<a href="https://colab.research.google.com/github/parikshithsivakumar/NLP/blob/main/twitter_roberta_base_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import accuracy_score, f1_score

# Load the dataset
data_path = 'Mock_Public_Sentiment_Dataset__1000_rows_.csv'  # Update this with your file path
df = pd.read_csv(data_path)

# Select relevant columns and drop missing values
df = df[['Post/Response Text', 'Sentiment']].dropna()

# Map sentiments to numerical labels
label_mapping = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df['Sentiment'] = df['Sentiment'].map(label_mapping)

# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Post/Response Text'], df['Sentiment'], test_size=0.2, random_state=42
)

# Load the tokenizer for Twitter-RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

train_data = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
test_data = Dataset.from_dict({'text': test_texts.tolist(), 'label': test_labels.tolist()})

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Load the pre-trained Twitter-RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment', num_labels=3)

# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': acc, 'f1': f1}

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to="none"  # Disable W&B integration
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

...

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-1-b5d4c9915711>:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.104900,1.100908,0.335000,0.201067
2,1.084100,1.100054,0.350000,0.286139
3,1.086300,1.094625,0.400000,0.379224


Evaluation Results: {'eval_loss': 1.0946247577667236, 'eval_accuracy': 0.4, 'eval_f1': 0.37922383922383923, 'eval_runtime': 1.3372, 'eval_samples_per_second': 149.57, 'eval_steps_per_second': 9.722, 'epoch': 3.0}


In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Suppress WandB logging if not required
os.environ["WANDB_DISABLED"] = "true"

# Load Dataset
df = pd.read_csv("deforestation_sentiment_dataset.csv")
print(len(df))

# Split Dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['sentiment'].tolist(), test_size=0.2, random_state=42
)

# Tokenizer
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# Custom Dataset Class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

train_dataset = SentimentDataset(train_texts, train_labels)
val_dataset = SentimentDataset(val_texts, val_labels)

# Load Pre-trained Model
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment', num_labels=3)

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save at the end of each epoch
    run_name="twitter_roberta_deforestation_sentiment",  # Custom run name
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy="epoch",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the Model
trainer.train()

# Evaluate the Model
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)
f1 = f1_score(val_labels, pred_labels, average='weighted')
print("F1 Score:", f1)

print(classification_report(val_labels, pred_labels, target_names=['Negative', 'Neutral', 'Positive']))

# Save the Model
model.save_pretrained('./fine_tuned_twitter_roberta')
tokenizer.save_pretrained('./fine_tuned_twitter_roberta')


1098


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.128600,0.076172
2,0.092600,0.088901
3,0.102900,0.084878


F1 Score: 0.96823456038287
              precision    recall  f1-score   support

    Negative       0.94      0.99      0.96        79
     Neutral       0.98      0.97      0.98        66
    Positive       0.99      0.95      0.97        75

    accuracy                           0.97       220
   macro avg       0.97      0.97      0.97       220
weighted avg       0.97      0.97      0.97       220



('./fine_tuned_twitter_roberta/tokenizer_config.json',
 './fine_tuned_twitter_roberta/special_tokens_map.json',
 './fine_tuned_twitter_roberta/vocab.json',
 './fine_tuned_twitter_roberta/merges.txt',
 './fine_tuned_twitter_roberta/added_tokens.json')

In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# Suppress WandB logging
os.environ["WANDB_DISABLED"] = "true"

# Load Dataset
df = pd.read_csv("deforestation_sentiment_dataset.csv")

# Preprocess Dataset
df['sentiment'] = df['sentiment'].astype(int)

# Stratified Split for Training and Validation Sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['sentiment'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['sentiment']
)

# Load Tokenizer for Twitter-RoBERTa
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# Custom Dataset Class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=512)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx],
        }

# Create Datasets
train_dataset = SentimentDataset(train_texts, train_labels)
val_dataset = SentimentDataset(val_texts, val_labels)

# Load Pre-trained Twitter-RoBERTa Model
model = RobertaForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment",
    num_labels=3,  # Match the number of sentiment classes
    ignore_mismatched_sizes=True  # Allow resizing of the classifier layer
)

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.1,
    load_best_model_at_end=True,
    save_total_limit=1,
    report_to="none",  # Disable WandB
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the Model
trainer.train()

# Evaluate the Model
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)

# Calculate Metrics
f1 = f1_score(val_labels, pred_labels, average='weighted')
classification_report_data = classification_report(val_labels, pred_labels, target_names=['Negative', 'Neutral', 'Positive'])

# Print Metrics
print(f"Weighted F1 Score: {f1}")
print(classification_report_data)

# Save the Fine-Tuned Model
model_save_path = "fine_tuned_twitter_roberta_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model saved at: {model_save_path}")


<ipython-input-5-8a58bbd643c0>:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.120000,0.135027
2,0.157000,0.164732
3,0.145100,0.176898
4,0.069300,0.172945


Weighted F1 Score: 0.945865697641337
              precision    recall  f1-score   support

    Negative       0.89      0.97      0.93        74
     Neutral       0.97      0.93      0.95        73
    Positive       0.99      0.93      0.96        73

    accuracy                           0.95       220
   macro avg       0.95      0.95      0.95       220
weighted avg       0.95      0.95      0.95       220

Model saved at: fine_tuned_twitter_roberta_model
